In [1]:
import pandas as pd
import numpy as np
import networkx as nx
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points
from geopy.distance import distance, lonlat

In [ ]:
df = pd.read_table('incremental_annual_rates/incremental_annual_rates_USA.txt', sep='\s+', header=None)
df.head()

In [ ]:
df.columns = ['Lat', 'Long'] + [str(np.around(x,1)) for x in np.arange(4.6, 8.7, 0.1)]
df.drop(df.columns[42:], axis=1, inplace=True)
df.fillna(value=0.0, inplace=True)
df = df[df['4.6'] != 0.0]
df.rename(columns={'Lat':'Long', 'Long':'Lat'}, inplace=True)

In [ ]:
df = pd.read_csv('incremental_annual_rates/incremental_annual_rates_italy.csv')
df.head()

In [ ]:
df.to_csv('incremental_annual_rates/incremental_annual_rates_italy_without_zeros.csv', index=False)

In [ ]:
g = nx.read_gml('networks/italy.gml')

In [ ]:
def intensity_europe(M, R):
    h = 3.91
    D = np.sqrt(h**2 + R**2)
    return 1.621*M - 1.343 - 0.0086*(D-h) - 1.037*(np.log(D)-np.log(h))

In [ ]:
l = [tuple(d.values()) for d in g.edges[('Rome', 'Pescara', 0)]['points']['point']]
edge = LineString(l)
point = Point(10.8,48.3)
nearest_point = nearest_points(edge, point)[0]
nearest_point.coords[:]

In [ ]:
for e in g.edges:
    edge_points = [tuple(p.values()) for p in g.edges[e]['points']['point']]
    edge = LineString(edge_points)
    epicenter = Point(10.5,48.2)
    nearest_point = nearest_points(edge, epicenter)[0].coords[0]
    dist = distance(lonlat(*epicenter.coords[0]), lonlat(*nearest_point)).km
    intensity = intensity_europe(8.6, dist)
    print("%.1f" % intensity)

In [82]:
#df['Epicenter'] = list(zip(df.Lat, df.Long))
#df.insert(loc=0, column='Epicenter', value=list(zip(df.Lat, df.Long)))
df.set_index('Epicenter', inplace=True)

In [101]:
epicenters = df.index.to_numpy()
magnitudes = df.columns[2:].to_numpy()

(48.3, 10.8)

In [ ]:
I = np.zeros((g.number_of_edges(), len(epicenters), len(magnitudes)))
for e in g.edges:
    edge_points = [tuple(p.values()) for p in g.edges[e]['points']['point']]
    l = LineString(edge_points)
    for epicenter in epicenters:
        p = Point(epicenter)
        nearest_point = nearest_points(l, p)[0].coords[0]
        R = distance(epicenter.coords[0], lonlat(*nearest_point)).km
        for M in magnitudes:
            i = intensity_europe(np.float(M), R)
    
    print("%.1f" % intensity)

In [113]:
list(enumerate(g.edges))

[(0, ('Rome', 'Pescara', 0)),
 (1, ('Rome', 'Florence', 0)),
 (2, ('Rome', 'Civitavecchia', 0)),
 (3, ('Rome', 'Naples', 0)),
 (4, ('Pescara', 'Bari', 0)),
 (5, ('Pescara', 'Bologna', 0)),
 (6, ('Bari', 'Messina', 0)),
 (7, ('Bari', 'Naples', 0)),
 (8, ('Catania', 'Mazara del Vallo', 0)),
 (9, ('Catania', 'Messina', 0)),
 (10, ('Palermo', 'Mazara del Vallo', 0)),
 (11, ('Palermo', 'Messina', 0)),
 (12, ('Mazara del Vallo', 'Cagliari', 0)),
 (13, ('Pisa', 'Florence', 0)),
 (14, ('Pisa', 'Genoa', 0)),
 (15, ('Pisa', 'Civitavecchia', 0)),
 (16, ('Florence', 'Bologna', 0)),
 (17, ('Bologna', 'Venice', 0)),
 (18, ('Bologna', 'Milan', 0)),
 (19, ('Genoa', 'Turin', 0)),
 (20, ('Genoa', 'Monaco', 0)),
 (21, ('Monaco', 'Marseille', 0)),
 (22, ('Marseille', 'Lausanne', 0)),
 (23, ('Turin', 'Milan', 0)),
 (24, ('Lausanne', 'Berne', 0)),
 (25, ('Berne', 'Milan', 0)),
 (26, ('Milan', 'Treviso', 0)),
 (27, ('Venice', 'Udine', 0)),
 (28, ('Venice', 'Treviso', 0)),
 (29, ('Udine', 'Treviso', 0)),
 (30